<a href="https://colab.research.google.com/github/SEKAR147/IJCNN_Work_SK/blob/main/Copy_of_Blip2_pipeline3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Standard "Stable" environment for your GQA project
!pip install -q huggingface_hub>=0.25.0 transformers>=4.45.0
!pip install -q accelerate bitsandbytes timm
import torch
from PIL import Image
import json
import os
import shutil
from google.colab import files
from transformers import (
    AutoProcessor,
    AutoModelForZeroShotObjectDetection,
    Blip2Processor,
    Blip2ForConditionalGeneration,
    AutoModelForCausalLM,
    AutoTokenizer
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.6 MB/s eta 0:00:00


In [ ]:
from transformers import BitsAndBytesConfig

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# Configure 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4", # Highly recommended for better quality
    bnb_4bit_use_double_quant=True
)

# 1. Grounding DINO (Remains the same)
dino_id = "IDEA-Research/grounding-dino-tiny"
dino_processor = AutoProcessor.from_pretrained(dino_id)
dino_model = AutoModelForZeroShotObjectDetection.from_pretrained(dino_id).to(device)

# 2. BLIP-2 (Fixed loading)
blip_id = "Salesforce/blip2-opt-2.7b"
blip_processor = Blip2Processor.from_pretrained(blip_id)
blip_model = Blip2ForConditionalGeneration.from_pretrained(
    blip_id,
    quantization_config=quantization_config, # Pass the config here
    device_map="auto"
)


preprocessor_config.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [ ]:
!pip install -q datasets
from datasets import load_dataset
from itertools import islice

In [ ]:
import spacy
from tqdm import tqdm # For the 1k progress bar
import torch

# Load the small model (fastest for attribute extraction)
# Disable components we don't need (ner, lemmatizer) to speed it up further
nlp = spacy.load("en_core_web_sm", disable=["ner", "lemmatizer"])

In [ ]:
def run_pipeline_streaming(image, text_prompt, question_id, image_id, base_drive_path):
    # 1. Setup specific folder for THIS image
    q_dir = os.path.join(base_drive_path, question_id)
    crops_dir = os.path.join(q_dir, "crops")
    os.makedirs(crops_dir, exist_ok=True)

    # 2. Save Original Image
    image = image.convert("RGB")
    image.save(os.path.join(q_dir, "original.jpg"))

    # --- GLOBAL SETUP (Outside main loop) ---
    # Exhaustive GQA vocabulary
    # Expanded Sets to match GQA Ontology
    COLOR_SET = {
        "red", "blue", "green", "yellow", "black", "white", "silver", "gold", "orange",
        "pink", "purple", "brown", "gray", "grey", "tan", "turquoise", "beige", "maroon",
        "navy", "teal", "azure", "bronze", "copper", "lavender", "violet", "blonde",
        "light", "dark", "bright", "pale", "colorful", "multicolored"
    }

    MATERIAL_SET = {
        "wood", "metal", "plastic", "glass", "brick", "denim", "leather", "cloth", "stone",
        "concrete", "paper", "rubber", "wool", "silk", "velvet", "fabric", "cotton", "nylon",
        "steel", "aluminum", "iron", "chrome", "porcelain", "ceramic", "tile", "marble",
        "asphalt", "cardboard", "wicker", "vinyl", "suede", "fleece"
    }

    TEXTURE_SET = {
        "smooth", "rough", "shiny", "metallic", "soft", "hard", "fuzzy", "clear", "dull",
        "matte", "wet", "dry", "painted", "polished", "glossy", "checkered", "striped",
        "dotted", "patterned", "cracked", "dirty", "clean", "worn", "new", "old", "fuzzy",
        "hairy", "scratched", "wrinkled", "rusted", "transparent", "opaque"
    }

    SHAPE_SET = {
        "round", "square", "rectangular", "triangular", "oval", "flat", "curved", "pointed",
        "thick", "thin", "wide", "narrow", "large", "small", "tiny", "huge", "long", "short",
        "tall", "curvy", "straight", "bent", "circular", "spherical", "conical", "cylindrical"
    }

    CLOTHING_SET = {
        "shirt", "pants", "hat", "glasses", "jacket", "jeans", "dress", "shorts", "shoes",
        "t-shirt", "sweater", "suit", "tie", "skirt", "boots", "sneakers", "socks", "gloves",
        "scarf", "belt", "cap", "helmet", "uniform", "vest", "coat", "hoodie"
    }

    # --- PHASE 1: GROUNDING DINO ---
    inputs = dino_processor(images=image, text=text_prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = dino_model(**inputs)

    results = dino_processor.post_process_grounded_object_detection(
        outputs, inputs.input_ids,
        threshold=0.35, text_threshold=0.25,
        target_sizes=[image.size[::-1]]
    )[0]

    final_data = []

    # --- PHASE 2 & 3: OBJECT LOOP ---
    for i, (score, label, box) in enumerate(zip(results["scores"], results["labels"], results["boxes"])):
        box = [round(b, 2) for b in box.tolist()]
        crop = image.crop((box[0], box[1], box[2], box[3]))

        # Skip very small noise
        if crop.size[0] < 40 or crop.size[1] < 40:
            continue

        # SAVE THE CROP TO DRIVE
        crop_name = f"obj_{i}_{label}.jpg"
        crop.save(os.path.join(crops_dir, crop_name))

        # Initialize attributes for THIS object
        extracted_attributes = []
        actions = []
        obj_id = f"gqa_2026_{question_id}_obj_{i}"

        # --- PHASE 2: BLIP-2 ---
        if label.lower() == "person":
            # Primes the model for clothing and action descriptions
            vqa_prompt = f"Question: Describe the clothing and current activity of this person. Answer: This person is wearing"
        else:
            # Primes the model for physical attributes
            vqa_prompt = f"Question: What are the specific visual properties of this {label}? Describe its color and material. Answer: This {label} is"
        blip_inputs = blip_processor(images=crop, text=vqa_prompt, return_tensors="pt").to(device, torch.float16)

        with torch.no_grad():
            gen_ids = blip_model.generate(**blip_inputs, max_new_tokens=40, min_new_tokens=10, repetition_penalty=1.2)
            desc = blip_processor.batch_decode(gen_ids, skip_special_tokens=True)[0].strip()
            desc = desc.split("Answer:")[-1].strip()

        # --- PHASE 3: SPACY ---


        # --- UPDATED PHASE 3 (Inside main loop) ---
        doc = nlp(desc.lower())
        # Initialize categorized buckets for this object
        categorized_attr = {
            "color": [],
            "material": [],
            "texture": [],
            "shape": [],
            "action": [],
            "clothing": []
        }

        for token in doc:
            t_text = token.text
            # 1. Action/Pose: Capture Present Participle verbs (-ing)
            if token.pos_ == "VERB" and token.tag_ == "VBG":
                if t_text not in ["is", "being", "having", "pose", "wearing"]:
                    categorized_attr["action"].append(t_text)

            # 2. Categorize based on GQA Sets
            if t_text in COLOR_SET:
                categorized_attr["color"].append(t_text)
            if t_text in MATERIAL_SET:
                categorized_attr["material"].append(t_text)
            if t_text in TEXTURE_SET:
                categorized_attr["texture"].append(t_text)
            if t_text in SHAPE_SET:
                categorized_attr["shape"].append(t_text)
            if t_text in CLOTHING_SET:
                categorized_attr["clothing"].append(t_text)

        # Deduplicate all lists
        categorized_attr = {k: list(set(v)) for k, v in categorized_attr.items()}

        # Save once per object
        final_data.append({
            "question_id": question_id,
            "img_id": image_id,
            "obj_id": obj_id,
            "box": box,
            "object_type": label,
            "attributes": categorized_attr, # Labeled output
            "crop_path": os.path.join("crops", crop_name),
            "raw": desc
        })

    # Save the JSON for this specific image inside its folder
    with open(os.path.join(q_dir, "data.json"), "w") as f:
        json.dump(final_data, f, indent=4)

    # Cleanup memory
    del results, inputs, outputs
    return final_data

In [ ]:
import os
from google.colab import drive

# 1. Forcefully unmount anything currently attached
!fusermount -u /content/drive

# 2. Delete the local 'ghost' directory to make it empty
!rm -rf /content/drive

# 3. Create a fresh, empty directory
!mkdir /content/drive

# 4. Now try the mount again
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_path = "/content/drive/MyDrive/GQA_Research_2026_7"
os.makedirs(drive_path, exist_ok=True)

fusermount: failed to unmount /content/drive: Invalid argument
Mounted at /content/drive


In [ ]:
import torch
import json
import os
from itertools import islice
from datasets import load_dataset

# 2. Load the Dataset (STREAMING MODE)
dataset = load_dataset("lmms-lab/GQA", "val_balanced_instructions", streaming=True, split="val")

# Images split to fetch the actual pictures
image_dataset = load_dataset("lmms-lab/GQA", "val_balanced_images", streaming=True, split="val")

# 2. Create a small lookup for images (for 1000 questions)
# Since we are streaming, we can iterate through the image split once
print("Building image lookup table...")
image_lookup = {}
for img_ex in islice(image_dataset, 5000): # Larger range to ensure we find all IDs
    image_lookup[img_ex['id']] = img_ex['image']

all_final_results = []
num_questions = 1000
prompt = "person . clothing . bag . chair . table . house . boat . door . animal . car . sign . bottle . cup . food . plate ."

print(f"Starting stream for {num_questions} images...")

# 3. The Main Loop
for i, example in enumerate(islice(dataset, num_questions)):
    q_id = example['id']
    img_id = example['imageId']

    # CHECK: Does the image exist in our lookup?
    if img_id not in image_lookup:
        print(f"Skipping Q {q_id}: Image {img_id} not found in cache.")
        continue

    img = image_lookup[img_id] # Fetch the actual PIL Image


    # Process image (returns the list of objects/attributes)
    try:
        image_results = run_pipeline_streaming(img, prompt, question_id = q_id, image_id=img_id, base_drive_path= drive_path)
        all_final_results.extend(image_results)
    except Exception as e:
        print(f"Skipping image {i} due to error: {e}")

    # Checkpoint every 50 images
    if i % 50 == 0 and i > 0:
        checkpoint_file = os.path.join(drive_path, "results_checkpoint.json")
        with open(checkpoint_file, "w") as f:
            json.dump(all_final_results, f)
        print(f"Checkpoint saved at question {i}")

        # Vital for T4 GPU health
        torch.cuda.empty_cache()

# 4. Final Save to Drive
final_file = os.path.join(drive_path, "final_gqa_results.json")
with open(final_file, "w") as f:
    json.dump(all_final_results, f)

print(f"Done! All results are saved in {drive_path}")

Building image lookup table...
Starting stream for 1000 images...


/usr/local/lib/python3.12/dist-packages/transformers/models/grounding_dino/processing_grounding_dino.py:93: FutureWarning: The key `labels` is will return integer ids in `GroundingDinoProcessor.post_process_grounded_object_detection` output since v4.51.0. Use `text_labels` instead to retrieve string object names.
  warnings.warn(self.message, FutureWarning)


Checkpoint saved at question 50
Checkpoint saved at question 100
Checkpoint saved at question 150
Checkpoint saved at question 200
Checkpoint saved at question 250
Checkpoint saved at question 300
Checkpoint saved at question 350
Checkpoint saved at question 400
Checkpoint saved at question 450
Checkpoint saved at question 500
Checkpoint saved at question 550
Checkpoint saved at question 600
Checkpoint saved at question 650
Checkpoint saved at question 700
Checkpoint saved at question 750
Checkpoint saved at question 800
Checkpoint saved at question 850
Checkpoint saved at question 900
Checkpoint saved at question 950
Done! All results are saved in /content/drive/MyDrive/GQA_Research_2026_7


In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
import os
import json
import numpy as np

# Load the model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0]); yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2]); yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    return interArea / float(areaA + areaB - interArea + 1e-6)

def run_semantic_evaluation(root_dir, gt_path, threshold=0.50):
    print(f"Loading Ground Truth...")
    with open(gt_path, 'r') as f:
        gt_data = json.load(f)

    results = {"total_items": 0, "semantic_hits": 0, "exact_hits": 0}

    print("Starting Semantic Analysis...")
    for root, dirs, files in os.walk(root_dir):
        if "data.json" not in files: continue

        with open(os.path.join(root, "data.json"), 'r') as f:
            predictions = json.load(f)
        if not predictions: continue

        img_id = str(predictions[0].get('img_id'))
        if img_id not in gt_data: continue
        gt_objs = gt_data[img_id]['objects']

        for p_obj in predictions:
            p_bbox = p_obj.get('box') # From your 'final_data' structure
            if not p_bbox: continue

            # --- SPATIAL MATCHING (Fixes the NameError) ---
            best_iou = 0
            best_gt_attrs = []

            for g_id, g_obj in gt_objs.items():
                # GQA [x, y, w, h] -> [x1, y1, x2, y2]
                g_bbox = [g_obj['x'], g_obj['y'], g_obj['x'] + g_obj['w'], g_obj['y'] + g_obj['h']]
                iou = calculate_iou(p_bbox, g_bbox)
                if iou > best_iou:
                    best_iou = iou
                    best_gt_attrs = [a.lower() for a in g_obj.get('attributes', [])]


            # Only proceed if we actually matched an object (IoU > 0.5)
            if best_iou >= 0.5 and best_gt_attrs:
                pred_list = [a for sublist in p_obj['attributes'].values() for a in sublist]
                if not pred_list: continue

                # --- SEMANTIC MATCHING ---
                # 1. Check Exact Matches first (Fast)
                exact_matches = [p for p in pred_list if p.lower() in best_gt_attrs]
                results["exact_hits"] += len(exact_matches)

                # 2. Check remaining for Semantic Similarity
                remaining_preds = [p for p in pred_list if p.lower() not in best_gt_attrs]
                results["total_items"] += len(pred_list)

                if remaining_preds:
                    pred_embeddings = sbert_model.encode(remaining_preds, convert_to_tensor=True)
                    gt_embeddings = sbert_model.encode(best_gt_attrs, convert_to_tensor=True)
                    cosine_scores = util.cos_sim(pred_embeddings, gt_embeddings)

                    for i in range(len(remaining_preds)):
                        if torch.max(cosine_scores[i]) >= threshold:
                            results["semantic_hits"] += 1

                # Add exact hits to semantic total
                results["semantic_hits"] += len(exact_matches)

    # Final Report
    exact_acc = (results['exact_hits']/results['total_items'])*100 if results['total_items'] > 0 else 0
    sem_acc = (results['semantic_hits']/results['total_items'])*100 if results['total_items'] > 0 else 0

    print("\n" + "="*40)
    print("      SEMANTIC EVALUATION REPORT")
    print("="*40)
    print(f"Exact Match Accuracy:   {exact_acc:.2f}%")
    print(f"Semantic Match Accuracy: {sem_acc:.2f}% (Threshold: {threshold})")
    print(f"Total Attributes Scored: {results['total_items']}")
    print("="*40)

# Run it
# run_semantic_evaluation("/content/drive/MyDrive/GQA_Research_2026_3", "/content/drive/MyDrive/GQA_Research_2026_3/val_sceneGraphs.json")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
run_semantic_evaluation(drive_path, "/content/drive/MyDrive/GQA_Research_2026_3/val_sceneGraphs.json", 0.6)

Loading Ground Truth...
Starting Semantic Analysis...

      SEMANTIC EVALUATION REPORT
Exact Match Accuracy:   15.34%
Semantic Match Accuracy: 34.06% (Threshold: 0.6)
Total Attributes Scored: 4980
